# Contenidos

1. [Introducción](#Introducción)
    - Explicación general del código

2. [Importación de Paquetes](#Importación-de-Paquetes)
    - Importación de librerías

3. [Carga de Datos](#Carga-de-Datos)
    - Descripción de los Datos
    - Exploración Inicial

4. [Preprocesamiento de Datos](#Preprocesamiento-de-Datos)
    - Limpieza de Datos
    - Transformación de Variables Categóricas
    - Escalado de Variables Numéricas
    - Balanceo del Dataset con SMOTE

5. [División del Conjunto de Datos](#División-del-Conjunto-de-Datos)
    - División en Entrenamiento y Prueba

6. [Modelado](#Modelado)
    - Modelos Implementados
        - Regresión Logística
        - Random Forest
        - Máquina de Soporte Vectorial (SVM)
        - K-Nearest Neighbors (KNN)
        - XGBoost

7. [Evaluación de Modelos](#Evaluación-de-Modelos)
    - Métricas de Desempeño
    - Análisis Comparativo de Modelos
    - Selección del Mejor Modelo

8. [Visualización de Resultados](#Visualización-de-Resultados)
    - Comparación de Métricas
    - Gráficos de Rendimiento

9. [Conclusión y Recomendaciones](#Conclusión-y-Recomendaciones)

10. [Referencias](#Referencias)

# Introducción

Este código realiza el preprocesamiento de datos, implementa SMOTE para balancear el conjunto de datos, entrena varios modelos de Machine Learning, calcula métricas de desempeño, y genera una tabla y gráficos comparativos de los modelos:

### Explicación:

1. **Preprocesamiento de datos:**
   - Se eliminan columnas irrelevantes (`zipCodeOrigin`, `zipMerchant`).
   - Se transforman las columnas categóricas en valores numéricos con `LabelEncoder`.
   - Se escala el monto de las transacciones para normalizar los datos.

2. **Balanceo del conjunto de datos:**
   - Se utiliza SMOTE para balancear las clases en el conjunto de datos.

3. **Entrenamiento de modelos:**
   - Se entrenan cinco modelos: Regresión Logística, Bosques Aleatorios, Máquina de Soporte Vectorial, XGBoost, y K-Nearest Neighbors.

4. **Cálculo de métricas:**
   - Se calculan métricas de desempeño como `F1-Score`, `Recall`, `Precisión`, `AUC`, y `Exactitud`.

5. **Comparación de modelos:**
   - Se genera una tabla y gráficos para comparar el desempeño de los modelos y determinar el mejor.

### Salida esperada:
- Una tabla que clasifica los modelos de mejor a peor basado en `F1-Score`.
- Gráficos que ilustran la comparación de las métricas y el ranking de los modelos.

# Importación de Paquetes

In [2]:
# carga, lectura y procesamiento
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

# Modelado
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Metricas
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

# Visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns

# Carga de Datos

In [3]:
# Cargar los datos usando una ruta relativa desde notebooks
data = pd.read_csv('../data/banksim.csv')

# Mostrar las primeras filas para verificar la carga
data.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


**Descripción del conjunto de datos:**
Se tienen 9 variables y la variable objetivo:

| **Columna**       | **Descripción**                                                                                                         | **Tipo de Datos** | **Valores/Notas**                                                                                   |
|-------------------|-------------------------------------------------------------------------------------------------------------------------|-------------------|------------------------------------------------------------------------------------------------------|
| **Step**          | Representa el día desde el inicio de la simulación. La simulación tiene 180 pasos, lo que equivale a 6 meses virtuales.  | Numérico          | 1-180                                                                                               |
| **Customer**      | Identificador único para cada cliente.                                                                                  | Texto             | Ejemplo: 'C1093826151'                                                                               |
| **zipCodeOrigin** | Código postal del origen o fuente de la transacción.                                                                     | Numérico          | Código postal de 5 dígitos                                                                           |
| **Merchant**      | Identificador único del comerciante.                                                                                     | Texto             | Ejemplo: 'M348934600'                                                                                |
| **zipMerchant**   | Código postal del comerciante.                                                                                           | Numérico          | Código postal de 5 dígitos                                                                           |
| **Age**           | Edad categorizada del cliente.                                                                                           | Numérico          | 0: <= 18, 1: 19-25, 2: 26-35, 3: 36-45, 4: 46-55, 5: 56-65, 6: > 65, U: Desconocido                   |
| **Gender**        | Género del cliente.                                                                                                      | Texto             | E: Empresa, F: Femenino, M: Masculino, U: Desconocido                                                |
| **Category**      | Categoría de la compra realizada.                                                                                        | Texto             | Ejemplo: 'es_transportation', 'es_health', etc.                                                      |
| **Amount**        | Monto de la transacción realizada.                                                                                       | Numérico          | Ejemplo: 4.55                                                                                        |
| **Fraud**         | Variable objetivo que indica si la transacción es fraudulenta (1) o benigna (0).                                          | Binario           | 1: Fraudulenta, 0: No fraudulenta                                                                    |

# Preprocesamiento de Datos

In [4]:
## Eliminar columnas con un único valor (zipCodeOrigin y zipMerchant)
data.drop(columns=['zipcodeOri', 'zipMerchant'], inplace=True)

## Convertir variables categóricas en valores numéricos
label_encoders = {}
categorical_columns = ['customer', 'merchant', 'category', 'gender', 'age']

for column in categorical_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

## Escalar las variables numéricas
scaler = StandardScaler()
data[['amount']] = scaler.fit_transform(data[['amount']])

In [5]:
## Definir variables independientes y dependientes
X = data.drop(columns=['fraud'])  # Variables independientes
y = data['fraud']  # Variable objetivo

In [6]:
## Balancear el conjunto de datos con SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# División del Conjunto de Datos

In [7]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Modelado

In [ ]:
from sklearn.model_selection import GridSearchCV

# Modelos a evaluar
modelos = {
    "Regresión Logística": LogisticRegression(),
    "Bosques Aleatorios": RandomForestClassifier(verbose=1, class_weight="balanced"),
    "Máquina de Soporte Vectorial": SVC(probability=True),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "XGBoost": XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
}

# Hiperparámetros optimizados para mayor rapidez
hiperparametros = {
    "Regresión Logística": {"C": [0.1, 1], "solver": ["liblinear"]},  
    "Bosques Aleatorios": {"n_estimators": [50, 100], "max_depth": [10, None]},  
    "Máquina de Soporte Vectorial": {"C": [0.1, 1], "kernel": ["linear"]},  
    "K-Nearest Neighbors": {"n_neighbors": [3, 5], "weights": ["uniform"]},  
    "XGBoost": {"n_estimators": [50, 100], "learning_rate": [0.1]}  
}

# Lista para almacenar los mejores hiperparámetros y resultados
mejores_hiperparametros = []
resultados = []

for nombre, modelo in modelos.items():
    print(f"Entrenando: {nombre}")
    
    # Configurar GridSearchCV
    grid = GridSearchCV(
        estimator=modelo, 
        param_grid=hiperparametros[nombre], 
        scoring='f1', 
        cv=2,  # Reducir a 2 folds para rapidez
        n_jobs=-1  # Usar múltiples núcleos
    )
    
    # Entrenar el modelo
    grid.fit(X_train, y_train)
    
    # Obtener el mejor modelo y sus hiperparámetros
    mejor_modelo = grid.best_estimator_
    mejores_params = grid.best_params_
    mejores_hiperparametros.append({"Modelo": nombre, "Mejores Hiperparámetros": mejores_params})
    
    # Predicciones con el mejor modelo
    y_pred = mejor_modelo.predict(X_test)
    y_prob = mejor_modelo.predict_proba(X_test)[:, 1] if hasattr(mejor_modelo, "predict_proba") else None
    
    # Calcular métricas
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None
    accuracy = accuracy_score(y_test, y_pred)
    
    resultados.append({
        "Modelo": nombre,
        "Mejores Hiperparámetros": mejores_params,
        "Precisión": precision,
        "Recall": recall,
        "F1-Score": f1,
        "AUC": auc,
        "Exactitud": accuracy
    })

# Convertir resultados en DataFrame y ordenarlos
resultados_df = pd.DataFrame(resultados).sort_values(by="F1-Score", ascending=False)
mejores_hiperparametros_df = pd.DataFrame(mejores_hiperparametros)

Entrenando: Regresión Logística
Entrenando: Máquina de Soporte Vectorial


# Evaluación de Modelos

**Comparativa de las metricas de desempeño:**

In [ ]:
# Mostrar metricas de desempeño
resultados_df

In [ ]:
# Mostrar mejores hiperparámetros
mejores_hiperparametros_df

**Selección del mejor modelo:**

El mejor modelo es... porque ...


# Visualización de Resultados

In [ ]:
## Gráfico de barras para las métricas
plt.figure(figsize=(12, 7))
sns.barplot(data=resultados_df.melt(id_vars=["Modelo", "Mejores Hiperparámetros"], value_vars=["F1-Score", "Precisión", "Recall", "AUC"]),
            x="variable", y="value", hue="Modelo", palette="Blues")
plt.title("Comparación de Modelos por Métricas")
plt.ylabel("Valor")
plt.xlabel("Métricas")
plt.legend(title="Modelo")
plt.show()

In [ ]:
## Gráfico de ranking por F1-Score
plt.figure(figsize=(10, 6))
sns.barplot(data=resultados_df, x="F1-Score", y="Modelo", palette="Blues")
plt.title("Ranking de Modelos por F1-Score")
plt.xlabel("F1-Score")
plt.ylabel("Modelo")
plt.show()

# Conclusión y Recomendaciones

Redactar conclusión recomendando por que usar el modelo seleccionado

# Referencias

Lopez-Rojas, Edgar Alonso ; Axelsson, Stefan

Banksim: A bank payments simulator for fraud detection research Inproceedings

26th European Modeling and Simulation Symposium, EMSS 2014, Bordeaux, France, pp. 144–152, Dime University of Genoa, 2014, ISBN: 9788897999324.

https://www.researchgate.net/publication/265736405_BankSim_A_Bank_Payment_Simulation_for_Fraud_Detection_Research